In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('Ramazan 2024 v1.xlsx')

In [ ]:
df.head()

# Single variable things: Referrals, Collectors

In [ ]:
# Extract Referrals
referrals = df.iloc[1:, 0].unique().tolist()
referrals.pop()  # Remove last item
referrals

df_referrals = pd.DataFrame(data={"name": referrals})
df_referrals["referralId"] = df_referrals.index + 1
df_referrals.to_csv('seed_tables_data/referrals.csv', index=False)

In [ ]:
# Extract Collectors
collectors = df.iloc[1:, 1].unique().tolist()
collectors.pop()  # Remove last item
collectors
df_collectors = pd.DataFrame(data={"name": collectors})
df_collectors["collectorId"] = df_collectors.index + 1
df_collectors.to_csv('seed_tables_data/collectors.csv', index=False)

# Vendors - Extract from Donor/Vendor column for expenses

In [ ]:
# Get all vendors from expenses (negative amounts in Sadqa/Zakat columns)
vendors_sadqa = df.loc[df.iloc[:, 7].astype(str).str.startswith("-"), df.columns[2]].unique().tolist()
vendors_zakat = df.loc[df.iloc[:, 5].astype(str).str.startswith("-"), df.columns[2]].unique().tolist()

# Combine and get unique vendors
all_vendors = list(set(vendors_sadqa + vendors_zakat))
all_vendors = [v for v in all_vendors if pd.notna(v)]  # Remove NaN

df_vendors = pd.DataFrame(data={"vendorName": sorted(all_vendors)})
df_vendors.to_csv('seed_tables_data/vendors.csv', index=False)
print(f"Total vendors: {len(all_vendors)}")
df_vendors.head(10)

# Tables: Donations, Expenses, Payments

## Expenses - Sadqa

In [ ]:
# Extract Sadqa expenses (negative amounts in Sadqah column 7)
expenses_sadqa = df.loc[df.iloc[:, 7].astype(str).str.startswith("-"), [df.columns[i] for i in [3, 7, 2]]]
expenses_sadqa.columns = ["date", "amount", "vendorProj"]

# Convert negative amounts to positive
expenses_sadqa["amount"] = -expenses_sadqa["amount"].astype(int)

# Set vendor name and project name (both from vendorProj column)
expenses_sadqa["vendorName"] = expenses_sadqa["vendorProj"]
expenses_sadqa["project"] = expenses_sadqa["vendorProj"]

# Set other fields based on new schema - status is "Pending" (unpaid)
expenses_sadqa["description"] = ""
expenses_sadqa["status"] = "Pending"

expenses_sadqa.drop(columns=["vendorProj"], inplace=True)
expenses_sadqa.reset_index(drop=True, inplace=True)
print(f"Sadqa expenses: {len(expenses_sadqa)}")
expenses_sadqa.head()

## Expenses - Zakat

In [ ]:
# Extract Zakat expenses (negative amounts in Zakat column 5)
expenses_zakat = df.loc[df.iloc[:, 5].astype(str).str.startswith("-"), [df.columns[i] for i in [3, 5, 2]]]
expenses_zakat.columns = ["date", "amount", "vendorProj"]

# Convert negative amounts to positive
expenses_zakat["amount"] = -expenses_zakat["amount"].astype(int)

# Set vendor name and project name (both from vendorProj column)
expenses_zakat["vendorName"] = expenses_zakat["vendorProj"]
expenses_zakat["project"] = expenses_zakat["vendorProj"]

# Set other fields
expenses_zakat["description"] = ""
expenses_zakat["status"] = "Pending"

expenses_zakat.drop(columns=["vendorProj"], inplace=True)
expenses_zakat.reset_index(drop=True, inplace=True)
print(f"Zakat expenses: {len(expenses_zakat)}")
expenses_zakat.head()

## Combine all expenses

In [ ]:
# Combine Sadqa and Zakat expenses
expenses_df = pd.concat([expenses_sadqa, expenses_zakat], ignore_index=True)
expenses_df["transacId"] = expenses_df.index + 1

# Reorder columns to match new schema
expenses_table = expenses_df[["transacId", "date", "amount", "vendorName", "project", "description", "status"]]
expenses_table.to_csv('seed_tables_data/expenses.csv', index=False)
print(f"Total expenses: {len(expenses_table)}")
expenses_table.head(10)

## Payments - Now separate from expenses

In [ ]:
# Extract Sadqa payments (same data as expenses but formatted for payments)
payments_sadqa = df.loc[df.iloc[:, 7].astype(str).str.startswith("-"), [df.columns[i] for i in [3, 1, 7, 4, 2]]]
payments_sadqa.columns = ["date", "collector", "amount", "paymentMethod", "vendorProj"]

# Map collector names to IDs
payments_sadqa["collectorId"] = payments_sadqa["collector"].map(lambda x: collectors.index(x) + 1)
payments_sadqa["amount"] = -payments_sadqa["amount"].astype(int)
payments_sadqa["paymentMethod"] = payments_sadqa["paymentMethod"].map(lambda x: "Cash" if x == "Cash" else "Online")
payments_sadqa["vendorName"] = payments_sadqa["vendorProj"]
payments_sadqa["type"] = "Sadqa"
payments_sadqa["status"] = "Completed"

payments_sadqa.drop(columns=["vendorProj", "collector"], inplace=True)
payments_sadqa.reset_index(drop=True, inplace=True)
print(f"Sadqa payments: {len(payments_sadqa)}")
payments_sadqa.head()

In [ ]:
# Extract Zakat payments
payments_zakat = df.loc[df.iloc[:, 5].astype(str).str.startswith("-"), [df.columns[i] for i in [3, 1, 5, 4, 2]]]
payments_zakat.columns = ["date", "collector", "amount", "paymentMethod", "vendorProj"]

payments_zakat["collectorId"] = payments_zakat["collector"].map(lambda x: collectors.index(x) + 1)
payments_zakat["amount"] = -payments_zakat["amount"].astype(int)
payments_zakat["paymentMethod"] = payments_zakat["paymentMethod"].map(lambda x: "Cash" if x == "Cash" else "Online")
payments_zakat["vendorName"] = payments_zakat["vendorProj"]
payments_zakat["type"] = "Zakat"
payments_zakat["status"] = "Completed"

payments_zakat.drop(columns=["vendorProj", "collector"], inplace=True)
payments_zakat.reset_index(drop=True, inplace=True)
print(f"Zakat payments: {len(payments_zakat)}")
payments_zakat.head()

In [ ]:
# Combine all payments
payments_df = pd.concat([payments_sadqa, payments_zakat], ignore_index=True)
payments_df["paymentId"] = payments_df.index + 1

# Reorder columns to match new schema
payments_table = payments_df[["paymentId", "vendorName", "collectorId", "type", "amount", "date", "paymentMethod", "status"]]
payments_table.to_csv('seed_tables_data/payments.csv', index=False)
print(f"Total payments: {len(payments_table)}")
payments_table.head(10)

## Donations

In [ ]:
# Extract Sadqa donations (positive amounts in Sadqah column)
donations_sadqa = df.loc[~df.iloc[:, 7].astype(str).str.startswith("-") & df.iloc[:, 7].notna() & (df.iloc[:, 7] != ''), 
                         [df.columns[i] for i in [3, 7, 4, 2, 1]]]
donations_sadqa.columns = ["date", "amount", "paymentMethod", "donorName", "collector"]

# Filter out header rows and empty
donations_sadqa = donations_sadqa[donations_sadqa["amount"] != "Collection"]
donations_sadqa["amount"] = donations_sadqa["amount"].astype(int)
donations_sadqa = donations_sadqa[donations_sadqa["amount"] > 0]

# Map names to IDs
donations_sadqa["referralId"] = donations_sadqa["donorName"].map(lambda x: referrals.index(x) + 1 if x in referrals else 1)
donations_sadqa["collectorId"] = donations_sadqa["collector"].map(lambda x: collectors.index(x) + 1 if x in collectors else 1)
donations_sadqa["paymentMethod"] = donations_sadqa["paymentMethod"].map(lambda x: "Cash" if str(x) == "Cash" else "Online")
donations_sadqa["type"] = "Sadqa"
donations_sadqa["status"] = "Completed"
donations_sadqa["notes"] = ""

donations_sadqa.drop(columns=["collector"], inplace=True)
donations_sadqa.reset_index(drop=True, inplace=True)
print(f"Sadqa donations: {len(donations_sadqa)}")
donations_sadqa.head()

In [ ]:
# Extract Zakat donations (positive amounts in Zakat column)
donations_zakat = df.loc[~df.iloc[:, 5].astype(str).str.startswith("-") & df.iloc[:, 5].notna() & (df.iloc[:, 5] != ''), 
                         [df.columns[i] for i in [3, 5, 4, 2, 1]]]
donations_zakat.columns = ["date", "amount", "paymentMethod", "donorName", "collector"]

# Filter out header rows
donations_zakat = donations_zakat[donations_zakat["amount"] != "Collection"]
donations_zakat["amount"] = donations_zakat["amount"].astype(int)
donations_zakat = donations_zakat[donations_zakat["amount"] > 0]

donations_zakat["referralId"] = donations_zakat["donorName"].map(lambda x: referrals.index(x) + 1 if x in referrals else 1)
donations_zakat["collectorId"] = donations_zakat["collector"].map(lambda x: collectors.index(x) + 1 if x in collectors else 1)
donations_zakat["paymentMethod"] = donations_zakat["paymentMethod"].map(lambda x: "Cash" if str(x) == "Cash" else "Online")
donations_zakat["type"] = "Zakat"
donations_zakat["status"] = "Completed"
donations_zakat["notes"] = ""

donations_zakat.drop(columns=["collector"], inplace=True)
donations_zakat.reset_index(drop=True, inplace=True)
print(f"Zakat donations: {len(donations_zakat)}")
donations_zakat.head()

In [ ]:
# Combine all donations
donations_df = pd.concat([donations_sadqa, donations_zakat], ignore_index=True)
donations_df["transacId"] = donations_df.index + 1

# Reorder columns
donations_table = donations_df[["transacId", "date", "amount", "donorName", "referralId", "collectorId", "type", "status", "notes", "paymentMethod"]]
donations_table.to_csv('seed_tables_data/donations.csv', index=False)
print(f"Total donations: {len(donations_table)}")
donations_table.head(10)

In [ ]:
print("\n=== Summary ===")
print(f"Referrals: {len(df_referrals)}")
print(f"Collectors: {len(df_collectors)}")
print(f"Vendors: {len(df_vendors)}")
print(f"Expenses: {len(expenses_table)}")
print(f"Payments: {len(payments_table)}")
print(f"Donations: {len(donations_table)}")